In [6]:
import cv2
import numpy as np

алгоритм Хорна-Шанка

In [60]:
def horn_schunck(I1, I2, alpha=0.001, num_iterations=100):
    I1 = I1.astype(np.float32)
    I2 = I2.astype(np.float32)

    # Compute gradients
    Ix = cv2.Sobel(I1, cv2.CV_32F, 1, 0, ksize=5)
    Iy = cv2.Sobel(I1, cv2.CV_32F, 0, 1, ksize=5)
    It = I2 - I1

    u = np.zeros(I1.shape, dtype=np.float32)
    v = np.zeros(I1.shape, dtype=np.float32)

    kernel = np.array([[1/12, 1/6, 1/12], [1/6, 0, 1/6], [1/12, 1/6, 1/12]])

    for _ in range(num_iterations):
        u_avg = cv2.filter2D(u, -1, kernel)
        v_avg = cv2.filter2D(v, -1, kernel)
        P = (Ix * u_avg + Iy * v_avg + It) / (alpha**2 + Ix**2 + Iy**2)
        u = u_avg - Ix * P
        v = v_avg - Iy * P

    return u, v

In [61]:
def draw_flow(img, u, v, step=16):
    h, w = img.shape[:2]
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = u[y, x], v[y, x]
    lines = np.vstack([x, y, x + fx, y + fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines)
    vis = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    for (x1, y1), (x2, y2) in lines:
        cv2.line(vis, (x1, y1), (x2, y2), (0, 255, 0), 1)
        cv2.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
    return vis

In [62]:
def process_video(video_path, alpha, num_iterations):
    cap = cv2.VideoCapture(video_path)
    
    ret, frame1 = cap.read()
    if not ret:
        print("Failed to read video")
        return
    prev_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    
    while True:
        ret, frame2 = cap.read()
        if not ret:
            break
        
        gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        
        # Calculate optical flow using Horn-Schunck method
        u, v = horn_schunck(prev_gray, gray, alpha=alpha, num_iterations=num_iterations)
        
        # Draw the optical flow on the frame
        flow_vis = draw_flow(gray, u, v)

        # Display the resulting frame
        cv2.imshow('Optical Flow', flow_vis)
        
        prev_gray = gray
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [63]:
video_path = '../assets/cars.mp4'
alpha_values = [1.0, 2.5]
num_iterations = 10

for alpha in alpha_values:
    print(f"Processing with alpha = {alpha}")
    process_video(video_path, alpha, num_iterations)

Processing with alpha = 1.0


KeyboardInterrupt: 

: 